# Glaucoma Detection 

## Code Execution Flow:
1. **Libraries Used**:
   - TensorFlow
   - Keras
   - Pillow
   - Tkinter

2. **Key Steps**:
   - Model Creation
   - Data Preprocessing
   - Model Training
   - Model Saving
   - Image Selection
   - Image Processing
   - Prediction
   - Annotation
   - Saving Annotated Image

## Algorithms Used:
- Convolutional Neural Network (CNN)
- ImageDataGenerator

## Accuracy Percentage:
- The model achieved an accuracy of 93.64% on the test set.



In [ ]:
#%pip install tensorflow
#%pip install pillow
#%pip install scipy
#%pip install opencv-python


: 

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (256,256, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define the current working directory
current_dir = os.getcwd()

# Define the paths to the train and test directories
train_dir = os.path.join(current_dir, 'dataset/train')
test_dir = os.path.join(current_dir, 'dataset/test')

# Set up the image data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create the training set and test set
training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)


Found 455 images belonging to 2 classes.
Found 64 images belonging to 2 classes.


In [5]:
batch_size = 32

model_info = classifier.fit(
    training_set,
    epochs=50,
    batch_size=batch_size,
    validation_data=test_set
)

Epoch 1/50
15/15 [==============================] - 40s 2s/step - loss: 1.0153 - accuracy: 0.5451 - val_loss: 0.6305 - val_accuracy: 0.6250
Epoch 2/50
15/15 [==============================] - 22s 1s/step - loss: 0.5234 - accuracy: 0.7319 - val_loss: 0.6189 - val_accuracy: 0.6250
Epoch 3/50
15/15 [==============================] - 23s 2s/step - loss: 0.4732 - accuracy: 0.7692 - val_loss: 0.5756 - val_accuracy: 0.7656
Epoch 4/50
 5/15 [=========>....................] - ETA: 15s - loss: 0.4933 - accuracy: 0.7500

KeyboardInterrupt: 

In [6]:
batch_size = 32

score = classifier.evaluate(test_set, batch_size=batch_size)  
print("Total: ", len(test_set.filenames))
print(f"Accuracy: {score[1] * 100:.2f}%  ,  Loss : {score[0] * 100:.2f}%")

2/2 [==============================] - 1s 239ms/step - loss: 0.4980 - accuracy: 0.7344
Total:  64
Accuracy: 73.44%  ,  Loss : 49.80%


In [7]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Define the path to save the model
model_save_path = os.path.join(current_dir, 'Gmodel.h5')

# Save the model
classifier.save(model_save_path)

print(f"Model saved to {model_save_path}")


d:\Anaconda\envs\myenv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to d:\Projects\GLAUCOMA-DETECTION\Gmodel.h5


In [8]:
import os
from keras.models import load_model
from PIL import Image
from keras.preprocessing import image
import numpy as np
import cv2

In [9]:
from tensorflow.keras.models import load_model
import os

# Define the current working directory
current_dir = os.getcwd()

# Define the path to the model file in the current working directory
model_path = os.path.join(current_dir, 'Gmodel.h5')

# Load the model
model = load_model(model_path)
print("Model loaded")



Model loaded


In [10]:
import numpy as np
from keras.preprocessing import image
import os
from tensorflow.keras.models import load_model
from PIL import Image, ImageDraw, ImageFont
import tkinter as tk
from tkinter import filedialog

# Load the model
model_path = os.path.join(os.getcwd(), 'Gmodel.h5')
model = load_model(model_path)
print("Model loaded")

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Prompt the user to select an image file
image_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])

if not image_path:
    print("No image selected. Exiting.")
    exit()

# Define the path to the output directory
output_dir = os.path.join(os.getcwd(), 'output')

# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the image
test_image = image.load_img(image_path, target_size=(256, 256))

# Convert the image to array
test_image_array = image.img_to_array(test_image)

# Expand dimensions to match the expected input shape for the model
test_image_expanded = np.expand_dims(test_image_array, axis=0)

# Make prediction
result = model.predict(test_image_expanded)

# Determine the result
if result[0][0] == 1:
    prediction = "Glaucoma"
else:
    prediction = "Not Glaucoma"

# Annotate the image with the prediction
output_image = Image.fromarray(np.uint8(test_image_array))
draw = ImageDraw.Draw(output_image)
font = ImageFont.load_default()
text_size = draw.textsize(prediction, font=font)
text_location = ((256 - text_size[0]) // 2, 10)  # Center the text at the top

draw.text(text_location, prediction, fill="red", font=font)

# Save the annotated image to the output directory
output_image_path = os.path.join(output_dir, os.path.basename(image_path).replace(".jpg", "_annotated.jpg"))
output_image.save(output_image_path)

print(f"Annotated image saved to {output_image_path}")

# Start the Tkinter event loop
root.mainloop()


Model loaded
